In [ ]:
#Inserted JSON data to MongoDB(Unemployment dataset)


# Reading unemployment-statistics data
unemployment_statistics_url = "https://data.ny.gov/api/views/dh9m-5v4d/rows.json?accessType=DOWNLOAD"
unemployment_statistics_response = urllib.request.urlopen(unemployment_statistics_url)
unemployment_statistics_data = json.loads(unemployment_statistics_response.read())

# inserting data into mongoDB
unemployment_statistics_collection = project_db["unemployment_statistics"]
# deleting extising data
unemployment_statistics_collection.drop()

cols = unemployment_statistics_data["meta"]["view"]["columns"]
cols_without_minus_one = filter(lambda val: val["id"] != -1, cols)
unemployment_statistics_cols = list(cols_without_minus_one)

unemployment_statistics_cols_others_len = len(list(filter(lambda val: val["id"] == -1, cols)))

for row in unemployment_statistics_data["data"]:
    db_row = {}
    for col_meta in unemployment_statistics_cols:
        data_value_index = int(col_meta["position"]) + unemployment_statistics_cols_others_len - 1
        db_row[col_meta["fieldName"]] = row[data_value_index]
    unemployment_statistics_collection.insert_one(db_row)

print("Inserted unemployment_statistics document count is; ", unemployment_statistics_collection.count_documents({}))

In [ ]:
# creating table in postgress
conn = psycopg2.connect(database="postgres",
                        host="localhost",
                        user="postgres",
                        password="admin",
                        port="5432")

cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS unemployment_statistics;CREATE TABLE unemployment_statistics (unemployment_statistics_id serial PRIMARY KEY, date_of_interest TIMESTAMP NOT NULL, area VARCHAR(255), labor_force BIGINT, employed BIGINT, unemployed BIGINT, unemployment_rate REAL);")
cursor.execute("DROP TABLE IF EXISTS covid_cases;CREATE TABLE covid_cases (covid_cases_id serial PRIMARY KEY, date_of_interest TIMESTAMP NOT NULL, case_count INTEGER);")
cursor.execute("DROP TABLE IF EXISTS air_quality;CREATE TABLE air_quality (air_quality_id serial PRIMARY KEY, date_of_interest TIMESTAMP NOT NULL, data_value REAL);")

conn.commit()

In [ ]:
# inserted values in unemployment table
unemployment_statistics_collection = project_db["unemployment_statistics"]
query = """INSERT INTO unemployment_statistics 
    (date_of_interest, area, labor_force, employed, unemployed, unemployment_rate) 
    VALUES (%s,%s,%s,%s,%s,%s)"""
cursor = conn.cursor()
for row in unemployment_statistics_collection.find():
    date = row["year"]+"-"+row["month"]+"-01"
    cursor.execute(query, (date, row["area"], row["labor_force"], row["employed"], 
                           row["unemployed"], row["unemployment_rate"]))
cursor.close()

In [ ]:
# Plots using seaborn library (1st plot)
cursor = conn.cursor();
query = """
SELECT date_of_interest, AVG(unemployment_rate)
    FROM unemployment_statistics 
    WHERE date_of_interest >= '2020-01-01' and date_of_interest < '2021-01-01'
    GROUP BY date_of_interest ORDER BY date_of_interest ASC 

"""

    
cursor.execute(query)
unemployment_rate_data = cursor.fetchall()
cursor.close()

    
    
unemployment_rate_df = pd.DataFrame(data = unemployment_rate_data,
                                   columns=["Date",
                                            "Unemployment Rate"])
# x_dates = unemployment_rate_df['Date'].dt.strftime('%Y-%m-%d').sort_values().unique()

fig, ax = plt.subplots(figsize = (12,6))

fig = sns.lineplot(
    data=unemployment_rate_df, 
    x="Date", 
    y="Unemployment Rate",
    estimator=sum,
    ax=ax
)

